## Parse data files from PushShift.io for selected subreddits using pyWren

In [12]:
import json
import bz2
import lzma
import pywren

TypeError: an integer is required (got type bytes)

In [7]:
data = []
for line in open('reddit_sample_data.json', 'r'):
    data.append(json.loads(line))

In [9]:
print(data[0])

{'author': 'Dethcola', 'author_flair_css_class': '', 'author_flair_text': 'Clairemont', 'body': 'A quarry', 'can_gild': True, 'controversiality': 0, 'created_utc': 1506816000, 'distinguished': None, 'edited': False, 'gilded': 0, 'id': 'dnqik14', 'is_submitter': False, 'link_id': 't3_73ieyz', 'parent_id': 't3_73ieyz', 'permalink': '/r/sandiego/comments/73ieyz/best_place_for_granite_counter_tops/dnqik14/', 'retrieved_on': 1509189606, 'score': 3, 'stickied': False, 'subreddit': 'sandiego', 'subreddit_id': 't5_2qq2q'}


In [10]:
target_subreddits = ['TheRedPill']

In [11]:
# comment parser for single file (month)
# Code adapted from: https://github.com/AhmedSoli/Reddit-Politics/blob/master/01_Content_Analysis/PreProcessing/.ipynb_checkpoints/010_ExtractCommentsTextCorpus-checkpoint.ipynb
def extract_comments(file_path):
    '''
    '''
    month_comments = {}
    for sub in target_subreddits:
        month_comments[sub] = {}
    if '.bz2' in file_path:
        o = bz2.open
    else:
        o = lzma.open

    with o(file_path,'rt') as comments_info:
            for i,comment in enumerate(comments_info):
                # load comment into json object
                comment = json.loads(comment)
#                 # check if comment contains information about where it was posted
#                 # all comments should include this information
#                 if 'subreddit' in comment:
#                     # check if the subreddit where the comment was posted
#                     # is in the target boards
                if comment['subreddit'] in target_subreddits:
                    # if yes append the body of the comment
                    month_comments[comment['subreddit']].append(comment['body']) #TODO: Add 'created_utc' ?
                
    # serlialize the result for this month
    pickle_out = open("reddit_comments/comments_corpus_" + file_path[:-4] + ".pickle","wb")
    pickle.dump(month_comments, pickle_out)
    pickle_out.close()
    

In [ ]:
# Parallelize comment scraping using pywren.
# Can either automize file search in order to store multiple file path names or just add them to iterdata list

pushshift_files = ['RC_2016_02.bz2']
pwex = pywren.default_executor()
futures = pwex.map(func = extract_comments, iterdata = pushshift_files)

monthly_files = pywren.get_all_results(futures) 

In [ ]:
comments_corpus = {}
# Keep file pickleing for storage reasons. 
# Storing all monthly comments into one corpus for each individual subreddit
for file in sorted(push_shift):
    comments_corpus_temp = pickle.load(open("../../Results/comments_corpus_" + file[:-4] + ".pickle","rb")) # loading in monthly files without file type
    for subreddit in comments_corpus_temp:
        if subreddit not in comments_corpus: # some selected subreddits might not be present in files
            comments_corpus[subreddit] = []
        # Add comment to entire corpus of comments for a single subreddit. This might be why we want to collect utc, incase we want to run time series on the data to map key events
        comments_corpus[subreddit].extend(
            comments_corpus_temp[subreddit]
        )
        print(subreddit,len(comments_corpus[subreddit]))
                        
pickle_out = open("reddit_comments/comments_corpus.pickle","wb")
pickle.dump(comments_corpus, pickle_out)
pickle_out.close()